In [13]:
import pandas as pd
import numpy as np

RAMDOM_STATE = 11

In [14]:
df = pd.read_csv('https://raw.githubusercontent.com/VitorBonella/PL-Dataset/main/dataset.csv',sep=";") #Leitura dos dados para o pandas

In [15]:
df.set_index('id',inplace=True) #Transformando a coluna id no indice da tabela

#Observem que a classe esta separada em duas colunas então devemos concatenadas para formar uma coluna só chamada classe
df['classe'] = df['tipo_lampada'].str.replace(" ", "") + df['potencia'].astype(str) 

In [16]:
# Lista de Indices
HU = ['i1', 'i2', 'i3', 'i4','i5', 'i6', 'i7']
MY = HU

df[MY] = df[MY].apply(lambda x: x.str.replace(',', '.').astype(float), axis=1)

In [51]:
results_df = pd.DataFrame(columns=['Método', 'Média', 'Desvio padrão', 'Limite inf.', 'Limite sup.'])

### zeroR

In [29]:
from sklearn.model_selection import cross_val_predict 
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from scipy import stats

zR = DummyClassifier(strategy='most_frequent')

X = df[MY]
Y = df['classe']

cv_zR = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=RAMDOM_STATE)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=RAMDOM_STATE)

scores = cross_val_score(zR, X_train, y_train, scoring='accuracy', cv=cv_zR)

inf, sup = stats.norm.interval(0.95, loc=scores.mean(), scale=scores.std()/np.sqrt(len(scores)))

results_df = pd.concat([results_df, pd.DataFrame({'Método': ['ZR'], 'Média': [scores.mean()], 'Desvio padrão': scores.std(), 'Limite inf.': inf, 'Limite sup.': sup})], ignore_index=True)

print(scores.mean())
print(type(scores))

0.1563405797101449
<class 'numpy.ndarray'>


In [19]:
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def train_models(model, params_grid):
    # Definir o número de rodadas para os ciclos de validação e teste
    rounds = 3

    # Criar o objeto RepeatedStratifiedKFold para os ciclos de validação
    inner_cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=rounds, random_state=RAMDOM_STATE)

    # Criar o objeto RepeatedStratifiedKFold para os ciclos de teste
    outer_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=rounds, random_state=RAMDOM_STATE)

    scalar = StandardScaler()
    pipe = Pipeline(steps=[('s',scalar), ('m', model)])
    # Criar o objeto GridSearchCV

    grid_search = GridSearchCV(pipe, param_grid=params_grid, cv=inner_cv)

    # Realizar os ciclos de validação e teste
    inner_scores = []
    outer_scores = []

    for _ in range(rounds):
        # Dividir os dados em conjunto de treinamento e teste externos
        X_train_outer, X_test_outer, y_train_outer, y_test_outer = train_test_split(X, Y, test_size=0.2, random_state=RAMDOM_STATE)

        # Realizar a busca em grade nos ciclos internos de validação
        grid_search.fit(X_train_outer, y_train_outer)

        # Obter o modelo com os melhores hiperparâmetros encontrados
        best_model = grid_search.best_estimator_

        # Avaliar o desempenho do modelo nos ciclos internos de validação
        inner_scores.extend(grid_search.cv_results_['mean_test_score'])

        # Avaliar o desempenho do modelo nos ciclos externos de teste
        outer_scores.append(best_model.score(X_test_outer, y_test_outer))

    # Calcular a pontuação média para os ciclos internos de validação
    inner_score_avg = np.mean(inner_scores)

    # Calcular a pontuação média para os ciclos externos de teste
    outer_score_avg = np.mean(outer_scores)

    return inner_score_avg, outer_score_avg


In [20]:
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy import stats

def train_models_2(model, params_grid, name):
    scalar = StandardScaler()
    pipe = Pipeline(steps=[('s',scalar), ('m', model)])

    inner_cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=RAMDOM_STATE)

    outer_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=RAMDOM_STATE)

    grid_search = GridSearchCV(pipe, param_grid=params_grid, cv=inner_cv, n_jobs=-1)

    scores = cross_val_score(grid_search, X, Y, scoring='accuracy', cv=outer_cv, n_jobs=-1)

    inf, sup = stats.norm.interval(0.95, loc=scores.mean(), scale=scores.std()/np.sqrt(len(scores)))

    local_df = pd.concat([results_df, pd.DataFrame({'Método': [name], 'Média': [scores.mean()], 'Desvio padrão': scores.std(), 'Limite inf.': inf, 'Limite sup.': sup})], ignore_index=True)

    return local_df, scores

In [21]:
from sklearn.ensemble import BaggingClassifier


bg = BaggingClassifier(n_estimators=3, random_state=RAMDOM_STATE)

params_grid = {
    'm__n_estimators': [3,9,15,21]
    } 

inner_score_avg, outer_score_avg = train_models(bg,params_grid)

results_df, scores_BA = train_models_2(bg, params_grid, 'BA')

print(inner_score_avg)
print(outer_score_avg)

0.4052848399246705
0.55


In [22]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators=3, random_state=RAMDOM_STATE)

params_grid = {
    'm__n_estimators': [3,9,15,21]
    }

inner_score_avg, outer_score_avg = train_models(ada,params_grid)

results_df, scores_AD = train_models_2(ada, params_grid, 'AD')

print(inner_score_avg)
print(outer_score_avg)

0.234504472693032
0.3


In [23]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(random_state=RAMDOM_STATE)

params_grid = {
    'm__n_estimators': [3,9,15,21]
} 

inner_score_avg, outer_score_avg = train_models(rf,params_grid)

results_df, score_RF = train_models_2(rf, params_grid, 'RF')

print(inner_score_avg)
print(outer_score_avg)

0.41054025423728807
0.5166666666666667


In [24]:
results_df

,Método,Média,Desvio padrão,Limite inf.,Limite sup.
0,ZR,0.156341,0.020735,0.148921,0.163761
1,BA,0.461571,0.096206,0.427145,0.495997
2,AD,0.257165,0.027720,0.247246,0.267084
3,RF,0.472797,0.074776,0.446039,0.499555


In [48]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.validation import check_X_y
from sklearn.utils import resample
from collections import Counter


class HeterogeneousPooling(BaseEstimator):
    # define o construtor para o classificador
    def __init__(self, n_samples=3):
        self.classifiers = [DecisionTreeClassifier(random_state=RAMDOM_STATE), KNeighborsClassifier(), GaussianNB()]
        self.n_samples = n_samples
        self.trained_classifiers = []

    def fit(self, X, y):
        X, y = check_X_y(X,y) #verifica os tamanhos
        
        self.counter = Counter(y)

        for i in range(self.n_samples):
            if i == 0:
                X_train = X
                y_train = y
            else:
                X_train, y_train = resample(X, y, replace=True, random_state=RAMDOM_STATE)

            self.train_classifiers(X_train, y_train)
    
    def train_classifiers(self, X, y):
        for classifier in self.classifiers:
            classifier.fit(X, y)
            self.trained_classifiers.append(classifier)

    def predict(self, X):

        predictions = []

        for _, row in X.iterrows():
            predictions.append(self.unique_predict(row.values.reshape(1, -1)))

        return np.ndarray(predictions)

    def unique_predict(self, X):
        predictions = {}

        for classifier in self.trained_classifiers:
            pred = classifier.predict(X)

            try:
                predictions[pred[0]] += 1
            except:
                predictions[pred[0]] = 1
        
        # verificar se existe mais de uma classe com a mesma quantidade de votos
        max_votes = max(predictions.values())
        max_classes = [k for k, v in predictions.items() if v == max_votes]

        if len(max_classes) > 1:
            # se sim, retornar a classe mais frequente na base de treino
            return self.counter.most_common(1)[0][0]
        else:
            # se não, retornar a classe mais votada
            return max_classes[0]

    def score(self, X, y):

        predictions = []

        for _, row in X.iterrows():
            predictions.append(self.unique_predict(row.values.reshape(1, -1)))

        return np.mean(predictions == y)

        

In [52]:
hp = HeterogeneousPooling()

params_grid = {
    'm__n_samples': [1,3,5,7]
}

# inner_score_avg, outer_score_avg = train_models(hp,params_grid)

results_df, scores_HP = train_models_2(hp, params_grid, 'HP')

In [53]:
results_df

,Método,Média,Desvio padrão,Limite inf.,Limite sup.
0,HP,NaN,NaN,NaN,NaN
